<a href="https://colab.research.google.com/github/HaTT2018/Air_Traffic/blob/main/Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Air Traffic Data and Import Basic Pkgs

In [ ]:
!pip install -U --no-cache-dir gdown --pre
!pip install ipdb

In [ ]:
!gdown --id 12UB_JFNcH2-FTNVgcCcGQD5CD0zYcLUP --no-cookies
!gdown --id 16wihT95GszfotQKv-a7xY6JEWk1abdI3 --no-cookies
!mkdir data
!mv 718598182_T_DB1B_MARKET.csv ./data/718598182_T_DB1B_MARKET.csv
!mv 'Smart Location Data Export.csv' './data/Smart Location Data Export.csv'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import ipdb

# Load Data

In [ ]:
data = pd.read_csv('./data/718598182_T_DB1B_MARKET.csv')

In [ ]:
data.head(2)

,YEAR,QUARTER,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_COUNTRY,ORIGIN_STATE_FIPS,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,...,DEST_COUNTRY,DEST_STATE_FIPS,DEST_STATE_ABR,DEST_STATE_NM,DEST_WAC,PASSENGERS,MARKET_FARE,MARKET_DISTANCE,NONSTOP_MILES,Unnamed: 24
0,2021,1,11503,1150305,31503,EGE,US,8,CO,Colorado,...,US,36,NY,New York,22,1.0,133.5,1746.0,1746.0,NaN
1,2021,1,12478,1247805,31703,JFK,US,36,NY,New York,...,US,8,CO,Colorado,82,1.0,134.0,1746.0,1746.0,NaN
